# Scraping Datasets of the Market History of all Counter Strike: Global Offensive Items

This notebook includes all the code written to scrape market history from steam json requests. I am still exploring/learning web scraping so the code may not be the best as I tried to experiment with new methods and technologies. Steam has multiple naming irregularities as well as rate limiting that made some manual data cleaning necessary. If you want to rescrape all data, change the values in the third cell of this notebook. Total data scraping time (excluding coding time) was approximately 12 hours. 

Data is split up into multiple xlsx files to account for potential crashes or errors in data collection (there were many).


In [1]:
import requests
import json
import pickle
from datetime import datetime, date
import time
import random
import pandas as pd
import numpy as np
import glob

In [17]:
# Settings Cell
appid = 730 #CSGO
currency = 1 # USD
country = "US"
cookie = {"steamLoginSecure": "PLACEHOLDER"} # steamloginsecure cookie. Please enter in your owns for this to work properly

FN = "Factory New"
MW = "Minimal Wear"
FT = "Field-Tested"
WW = "Well-Worn"
BS = "Battle-Scarred"

starting_date = date(2013, 8, 1) # originally scraped from 8-1-2013 but upon data analysis found the first date of market sales is 8-13-2013
ending_date = date(2020, 9, 14)

In [18]:
# Creates a list of dates in the date range for comparison to data pulled from the market later
dates_list = pd.date_range(start = starting_date, end = ending_date)

## get_history

get_history() is the primary function used to pull data from the Steam server using a json get request with the item name and symbols formatted to ASCII. Steam servers will return a status indicating whether the request was successful and a list of prices and the volume sold for each particular date. If a date sold nothing, there is no entry and the list simply skips that date. In order to combat this we use date_list, a list of every date in between our given range. Each date in the date list is compared to the date given in the steam data. If the steam data date does not match, we append a None, None to the price/volume list and increase the date_list index up again until the data has another value. 

This function underwent multiple revisions. Other possible variants include checking the difference between the current date and the previous date and appending Nones based on that value.

Steam also gives hourly? (unsure if hourly but seems so) price data for more recent dates. This means a date can appear multiple times. In order to address this, data with the same date as the previous data entry is aggregated together (price is averaged and volume is summed).  

In [13]:
def get_history(name):
    url = "http://steamcommunity.com/market/pricehistory/?country={0}&currency={1}&appid={2}&market_hash_name={3}".format(country, currency, appid, name)

    for x in range(3): # occasionally will hit connection errors
        try:
            item = requests.get(url, cookies = cookie)
        except Exception as e:
            print(e)
            if x == 2:
                return 1
            else:
                time.sleep(60) # waits 60 seconds to allow steam to refresh
                continue
        else:
            break

    item = item.content
    item = json.loads(item)

    if item: # checks if returned anything
        item_prices = item["prices"] 
        success = item["success"]
        if (item_prices == False) | (not item_prices) | (success == False):
            return 1
        else:
            print("{0} CURRENTLY SCRAPING ITEM: {1}, NUMBER {2} OF {3} ITEMS".format(datetime.now(), name, current_num, total_items)) # status check
            pricevol = [] # price and volume are combined for multiindexing in the dataframe later
            dates = [] # used to check whether dates repeat
            item_counter = 0 
            date_counter = -1

            while (date_counter < len(dates_list) - 1) & (item_counter < len(item_prices)):
                date_counter += 1
                current_date = datetime.strptime(item_prices[item_counter][0][0:11], "%b %d %Y").date()
                if (current_date == dates_list[date_counter - 1]): # some entries have multiple entries per day
                    pricevol[-2] = np.mean([item_prices[item_counter][1], pricevol[-2]]) # averages the prices in the day and rewrites the previously written one
                    pricevol[-1] = int(item_prices[item_counter][2]) + pricevol[-1] # adds the volumes for the day together
                    date_counter -= 1
                    item_counter += 1
                elif dates_list[date_counter] != current_date:
                    pricevol.extend((None, None))
                else:
                    pricevol.append(item_prices[item_counter][1]) # appending price
                    pricevol.append(int(item_prices[item_counter][2])) # appending volume
                    dates.append(current_date) # appending the date into a list
                    item_counter += 1
        
            if current_date != ending_date: # adds extra nones to end if the present has no sales
                diff = ending_date - current_date
                pricevol.extend([None for x in range(0, diff.days * 2)])
    else:
        print("No contents on {}".format(name))
        return 1
    return pricevol

## Weapon Skins Price History Collection

Scraping weapon skins involves formatting the skin's name and iterating through all possible conditions, souvenir, and StatTrak variants.


In [66]:
all_skins = pd.read_excel("items_list/skins_list.xlsx")

knives = pd.read_excel("items_list/knivesgloves_list.xlsx")
all_skins = all_skins.append(knives, ignore_index = True)

all_skins

,Weapon,Collection,Skin,Quality,StatTrak,Souvenir
0,CZ75-Auto,The Baggage Collection,Green Plaid,Consumer Grade,False,False
1,CZ75-Auto,The Chop Shop Collection,Army Sheen,Consumer Grade,False,False
2,CZ75-Auto,The Canals Collection,Indigo,Consumer Grade,False,False
3,CZ75-Auto,The Bank Collection,Tuxedo,Mil-Spec,False,False
4,CZ75-Auto,The Overpass Collection,Nitro,Mil-Spec,False,True
...,...,...,...,...,...,...
1293,Hand Wraps,Clutch Case,Cobalt Skulls,Extraordinary,False,False
1294,Driver Gloves,Clutch Case,Racing Green,Extraordinary,False,False
1295,Driver Gloves,Clutch Case,Overtake,Extraordinary,False,False
1296,Driver Gloves,Clutch Case,Imperial Plaid,Extraordinary,False,False


In [68]:
def make_hash_name(weapon, skin, condition, st, sv):
    if skin == "★ (Vanilla)":
        name = "{0}".format(weapon) # vanilla knives have no condition or skin name
    else:
        name = "{0} | {1} ({2})".format(weapon, skin, condition)
        
    if st == True:
        name = "StatTrak™ " + name
    elif sv == True: 
        name = "Souvenir " + name
        
    if ("Knife" in weapon) | ("Gloves" in weapon): # knives and gloves have a star before their name
        name = "★ " + name
        
    formatted_name = name.replace(" ", "%20") 
    formatted_name = formatted_name.replace("&", "%26")
    formatted_name = formatted_name.replace("|", "%7C")
    formatted_name = formatted_name.replace("+", "%2B")
    formatted_name = formatted_name.replace(":", "%3A")
    formatted_name = formatted_name.replace("/", "%2F")
    formatted_name = formatted_name.replace("(", "%28")
    formatted_name = formatted_name.replace(")", "%29")

    return formatted_name

In [69]:
skin_history = {}
i = 0
current_num = 0
total_items = len(all_skins.index)

for index, row in all_skins.iterrows():
    current_num += 1
    # checks whether to reiterate for stattrak and souvenir
    if (row["StatTrak"] == True) | (row["Souvenir"] == True):
        svst = ((False, False), (row["StatTrak"], row["Souvenir"]))
    else:
        svst = ((False, False),)
    for st, sv in svst:
        if row["Skin"] == "★ (Vanilla)":
            possible_conditions = (None,)
        else:
            possible_conditions = (BS, WW, FT , MW , FN)
        for condition in possible_conditions: # iterates through all possible conditions
            time.sleep(random.uniform(.5, 3)) # avoid steam rate limiting with a random sleep
            name = make_hash_name(row["Weapon"], row["Skin"], condition, st, sv)
            price_vol_history = get_history(name)
            if price_vol_history == 1:
                print("{0} HAS FAILED".format(name))
                continue
            col_index = "col_" + str(i)
            skin_history[col_index] = [row["Weapon"], row["Collection"], row["Quality"], row["Skin"], condition, st, sv] + price_vol_history
            i += 1
    if current_num % 100 == 0: # saves the data to csv every 100 items to protect against crashes
        print("made a new file!")
        skins_prices = pd.DataFrame.from_dict(skin_history, orient = "index")
        skins_prices.to_excel("skins_query/skins{}.xlsx".format(current_num / 100))
        skin_history.clear()

if skins_price != {}:
    print("finished!")
    skins_prices = pd.DataFrame.from_dict(skin_history, orient = "index")
    skins_prices.to_excel("skins_query/skins_final.xlsx")

KeyboardInterrupt: 

## Stickers Price History Collection

Sticker history collection works a bit different from skins. Sticker names have many variants depending on the collection or lack thereof. 

In [27]:
all_stickers = pd.read_excel("items_list/stickers_list.xlsx")

all_stickers

,Weapon,Collection,Skin,Quality
0,Sticker,Atlanta 2017,RpK,High Grade
1,Sticker,Atlanta 2017,ELEAGUE,High Grade
2,Sticker,Atlanta 2017,SmithZz,High Grade
3,Sticker,Atlanta 2017,TaZ,High Grade
4,Sticker,Atlanta 2017,byali,High Grade
...,...,...,...,...
3596,Sticker,"Warhammer 40,000 Sticker Capsule",Heresy (Holo),Remarkable
3597,Sticker,"Warhammer 40,000 Sticker Capsule",Bloodthirster (Foil),Exotic
3598,Sticker,"Warhammer 40,000 Sticker Capsule",Chaos Marine (Foil),Exotic
3599,Sticker,"Warhammer 40,000 Sticker Capsule",Emperor (Foil),Exotic


In [4]:
def make_sticker_hash(collection, skin):
    if collection == None:
        name = "Sticker | {0}".format(skin)
    elif "201" in collection & ("community", "winter") not in collection: # if there is a year in the name and is tourney capsule
        name = "Sticker | {0} | {1}".format(skin, collection)
    else:
        name = "Sticker | {0}".format(skin)

    formatted_name = name.replace(" ", "%20") 
    formatted_name = formatted_name.replace("&", "%26")
    formatted_name = formatted_name.replace("|", "%7C")
    formatted_name = formatted_name.replace("+", "%2B")
    formatted_name = formatted_name.replace(":", "%3A")
    formatted_name = formatted_name.replace("/", "%2F")
    formatted_name = formatted_name.replace("(", "%28")
    formatted_name = formatted_name.replace(")", "%29")

    return formatted_name

In [30]:
sticker_history = {}
i = 0
current_num = 0
total_items = len(all_stickers.index)

for index, row in all_stickers.iterrows():
    current_num += 1
    time.sleep(random.uniform(.5, 3)) # avoid steam rate limiting with a random sleep
    name = make_sticker_hash(row["Collection"], row["Skin"])
    price_vol_history = get_history(name)
    if price_vol_history == 1:
        print("{0} HAS FAILED".format(name))
        continue
    col_index = "col_" + str(i)
    sticker_history[col_index] = ["Sticker", row["Collection"], row["Quality"], row["Skin"], None, None, None] + price_vol_history
    i += 1
    if current_num % 300 == 0: # saves the data to csv every 100 items to protect against crashes
        print("made a new file!")
        skins_prices = pd.DataFrame.from_dict(sticker_history, orient = "index")
        skins_prices.to_excel("stickers_query/stickers{}.xlsx".format(current_num / 300))
        sticker_history.clear()

if sticker_history != {}:
    print("finished!")
    skins_prices = pd.DataFrame.from_dict(sticker_history, orient = "index")
    skins_prices.to_excel("stickers_query/stickers_final.xlsx")

2020-09-15 21:37:08.778037 CURRENTLY SCRAPING ITEM: Sticker%20%7C%20Retake%20Expert, NUMBER 1 OF 23 ITEMS
2020-09-15 21:37:12.371657 CURRENTLY SCRAPING ITEM: Sticker%20%7C%20Dessert%20Eagle, NUMBER 2 OF 23 ITEMS
2020-09-15 21:37:15.242572 CURRENTLY SCRAPING ITEM: Sticker%20%7C%20Devouring%20Flame, NUMBER 3 OF 23 ITEMS
2020-09-15 21:37:18.841021 CURRENTLY SCRAPING ITEM: Sticker%20%7C%20Entry%20Fragger, NUMBER 4 OF 23 ITEMS
2020-09-15 21:37:22.136268 CURRENTLY SCRAPING ITEM: Sticker%20%7C%20Bullet%20Rain, NUMBER 5 OF 23 ITEMS
2020-09-15 21:37:25.535727 CURRENTLY SCRAPING ITEM: Sticker%20%7C%20Camper, NUMBER 6 OF 23 ITEMS
2020-09-15 21:37:29.643892 CURRENTLY SCRAPING ITEM: Sticker%20%7C%20Friendly%20Fire, NUMBER 7 OF 23 ITEMS
2020-09-15 21:37:34.120177 CURRENTLY SCRAPING ITEM: Sticker%20%7C%20Small%20Arms, NUMBER 8 OF 23 ITEMS
2020-09-15 21:37:36.368169 CURRENTLY SCRAPING ITEM: Sticker%20%7C%20Retake%20Expert%20%28Holo%29, NUMBER 9 OF 23 ITEMS
2020-09-15 21:37:40.463310 CURRENTLY SCRAPING

## Cases Price History Collection

Cases work with a simple query. Very little changes to name needed.

In [87]:
all_cases = pd.read_excel("items_list/cases_list.xlsx")

all_cases

,Weapon,Collection,Skin
0,Patch Pack,Half-Life: Alyx Patch Pack,Half-Life: Alyx Patch Pack
1,Patch Pack,CS:GO Patch Pack,CS:GO Patch Pack
2,Music Kit Box,Masterminds Music Kit Box,Masterminds Music Kit Box
3,Must Kit Box,StatTrak™ Radicals Box,StatTrak™ Radicals Box
4,Pins Capsule,Collectible Pins Capsule Series 1,Collectible Pins Capsule Series 1
...,...,...,...
295,\nSouvenir Packages\n,The Dust 2 Collection,Berlin 2019 Dust II Souvenir Package
296,\nSouvenir Packages\n,The Overpass Collection,Berlin 2019 Overpass Souvenir Package
297,\nSouvenir Packages\n,The Train Collection,Berlin 2019 Train Souvenir Package
298,\nSouvenir Packages\n,The 2018 Nuke Collection,Berlin 2019 Nuke Souvenir Package


In [88]:
def make_case_hash(skin):
    name = skin

    formatted_name = name.replace(" ", "%20") 
    formatted_name = formatted_name.replace("&", "%26")
    formatted_name = formatted_name.replace("|", "%7C")
    formatted_name = formatted_name.replace("+", "%2B")
    formatted_name = formatted_name.replace(":", "%3A")
    formatted_name = formatted_name.replace("/", "-")
    formatted_name = formatted_name.replace("(", "%28")
    formatted_name = formatted_name.replace(")", "%29")

    return formatted_name

In [89]:
case_history = {}
i = 0
current_num = 0
total_items = len(all_cases.index)

for index, row in all_cases.iterrows():
    current_num += 1
    time.sleep(random.uniform(.5, 3)) # avoid steam rate limiting with a random sleep
    name = make_case_hash(row["Skin"])
    price_vol_history = get_history(name)
    if price_vol_history == 1:
        print("{0} HAS FAILED".format(name))
        continue
    col_index = "col_" + str(i)
    case_history[col_index] = [row["Weapon"], row["Collection"], None, row["Skin"], None, None, None] + price_vol_history
    i += 1
    if current_num % 300 == 0: # saves the data to csv every 100 items to protect against crashes
        print("made a new file!")
        skins_prices = pd.DataFrame.from_dict(case_history, orient = "index")
        skins_prices.to_excel("others_query/cases{}.xlsx".format(current_num / 300))
        case_history.clear()

if case_history != {}:
    print("finished!")
    skins_prices = pd.DataFrame.from_dict(case_history, orient = "index")
    skins_prices.to_excel("others_query/cases_final.xlsx")

2020-09-15 23:30:15.361245 CURRENTLY SCRAPING ITEM: Half-Life%3A%20Alyx%20Patch%20Pack, NUMBER 1 OF 300 ITEMS
2020-09-15 23:30:19.081588 CURRENTLY SCRAPING ITEM: CS%3AGO%20Patch%20Pack, NUMBER 2 OF 300 ITEMS
2020-09-15 23:30:22.783587 CURRENTLY SCRAPING ITEM: Masterminds%20Music%20Kit%20Box, NUMBER 3 OF 300 ITEMS
2020-09-15 23:30:26.413773 CURRENTLY SCRAPING ITEM: StatTrak™%20Radicals%20Box, NUMBER 4 OF 300 ITEMS
2020-09-15 23:30:28.783394 CURRENTLY SCRAPING ITEM: Collectible%20Pins%20Capsule%20Series%201, NUMBER 5 OF 300 ITEMS
2020-09-15 23:30:33.500172 CURRENTLY SCRAPING ITEM: Collectible%20Pins%20Capsule%20Series%202, NUMBER 6 OF 300 ITEMS
2020-09-15 23:30:38.084211 CURRENTLY SCRAPING ITEM: Collectible%20Pins%20Capsule%20Series%203, NUMBER 7 OF 300 ITEMS
2020-09-15 23:30:40.215229 CURRENTLY SCRAPING ITEM: Half-Life%3A%20Alyx%20Collectible%20Pins%20Capsule, NUMBER 8 OF 300 ITEMS
2020-09-15 23:30:41.844366 CURRENTLY SCRAPING ITEM: Community%20Graffiti%20Box%201, NUMBER 9 OF 300 ITEMS


## Others Price History Collection

Others are a bit more finnicky. Music Kits do have a StatTrak variant which much be accounted for by iterating through StatTraks. The various items are also formatted in wildly different ways ways which also must be accounted for.

In [19]:
all_others = pd.read_excel("items_list/others_list.xlsx")

all_others

,Weapon,Collection,Skin,Quality,StatTrak
0,Agents,Shattered Web Case,Soldier | Phoenix,Distinguished,False
1,Agents,Shattered Web Case,Ground Rebel | Elite Crew,Distinguished,False
2,Agents,Shattered Web Case,Enforcer | Phoenix,Distinguished,False
3,Agents,Shattered Web Case,Operator | FBI SWAT,Distinguished,False
4,Agents,Shattered Web Case,B Squadron Officer | SAS,Distinguished,False
...,...,...,...,...,...
1787,Items,NaN,Prisma Case Key,NaN,False
1788,Items,NaN,Berlin 2019 Viewer Pass,NaN,False
1789,Items,NaN,Berlin 2019 Viewer Pass + 3 Souvenir Tokens,NaN,False
1790,Items,NaN,CS20 Case Key,NaN,False


In [63]:
def make_others_hash(weapon, skin, st):
    if (weapon in ("Agents", "Items", "Collectible Pins")) | ("StatTrak" in skin): # certain items are formatted properly in list and dont need their weapon added to again
        name = skin
    else: 
        name = "{0} | {1}".format(weapon, skin)
    if st:
        name = "StatTrak™ " + name

    formatted_name = name.replace(" ", "%20") 
    formatted_name = formatted_name.replace("&", "%26")
    formatted_name = formatted_name.replace("|", "%7C")
    formatted_name = formatted_name.replace("+", "%2B")
    formatted_name = formatted_name.replace(":", "%3A")
    formatted_name = formatted_name.replace("/", "-")
    formatted_name = formatted_name.replace(",", "%2C")
    formatted_name = formatted_name.replace("(", "%28")
    formatted_name = formatted_name.replace(")", "%29")

    return formatted_name

In [65]:
other_history = {}
i = 0
current_num = 0
total_items = len(all_others.index)

for index, row in all_others.iterrows():
    current_num += 1
    if row["StatTrak"]:
        st_opt = (False, True)
    else:
        st_opt = (False,)
    
    for st in st_opt:
        time.sleep(random.uniform(.5, 3)) # avoid steam rate limiting with a random sleep
        name = make_others_hash(row["Weapon"], row["Skin"], st)
        price_vol_history = get_history(name)
        if price_vol_history == 1:
            print("{0} HAS FAILED".format(name))
            continue
        col_index = "col_" + str(i)
        other_history[col_index] = [row["Weapon"], row["Collection"], row["Quality"], row["Skin"], None, st, None] + price_vol_history
        i += 1
    if current_num % 300 == 0: # saves the data to csv every 100 items to protect against crashes
        print("made a new file!")
        skins_prices = pd.DataFrame.from_dict(other_history, orient = "index")
        skins_prices.to_excel("others_query/others{}.xlsx".format(current_num / 300))
        other_history.clear()

if other_history != {}:
    print("finished!")
    skins_prices = pd.DataFrame.from_dict(other_history, orient = "index")
    skins_prices.to_excel("others_query/others_final2.xlsx")

2020-09-16 03:05:14.284321 CURRENTLY SCRAPING ITEM: StatTrak™%20Music%20Kit%20%7C%20Neck%20Deep%2C%20Life's%20Not%20Out%20To%20Get%20You, NUMBER 1 OF 7 ITEMS
2020-09-16 03:05:16.429761 CURRENTLY SCRAPING ITEM: StatTrak™%20Music%20Kit%20%7C%20Blitz%20Kids%2C%20The%20Good%20Youth, NUMBER 2 OF 7 ITEMS
2020-09-16 03:05:20.101735 CURRENTLY SCRAPING ITEM: StatTrak™%20Music%20Kit%20%7C%20Roam%2C%20Backbone, NUMBER 3 OF 7 ITEMS
2020-09-16 03:05:23.210343 CURRENTLY SCRAPING ITEM: StatTrak™%20Music%20Kit%20%7C%20Twin%20Atlantic%2C%20GLA, NUMBER 4 OF 7 ITEMS
2020-09-16 03:05:30.569489 CURRENTLY SCRAPING ITEM: StatTrak™%20Music%20Kit%20%7C%20Hundredth%2C%20FREE, NUMBER 5 OF 7 ITEMS
2020-09-16 03:05:33.758967 CURRENTLY SCRAPING ITEM: StatTrak™%20Music%20Kit%20%7C%20Beartooth%2C%20Aggressive, NUMBER 6 OF 7 ITEMS
2020-09-16 03:05:36.254119 CURRENTLY SCRAPING ITEM: StatTrak™%20Music%20Kit%20%7C%20Skog%2C%20III-Arena, NUMBER 7 OF 7 ITEMS
finished!


## Quick Data Cleaning

A majority of the data cleaning occurs during analysis but before then a multilevel index is added to help organize all the dates. The following cells iterate through each file of chunked up files to create one big dataframe where an index is added and saved as an overarching xlsx file.

In [24]:
# create a multilevel index for organizing the days
f = np.repeat([d.strftime("%Y-%m-%d") for d in dates_list], 2)
first_tier = ["Weapon", "Collection", "Quality", "Skin","Condition", "StatTrak", "Souvenir"]
second_tier = ["" for x in range(len(first_tier))] + [y for x in range(2602) for y in ["Price", "Volume"]]
first_tier.extend(f)
index = pd.MultiIndex.from_arrays([first_tier, second_tier])

In [27]:
len(index)

5211

In [48]:
# turns all skin files into one big data frame
path = r"D:\Code\steam_market_tracker\skins_query"
all_files = glob.glob(path + "/*.xlsx")

li = []

for filename in all_files:
    df = pd.read_excel(filename, index_col = None, header = 0)
    li.append(df)

skins_history = pd.concat(li, axis = 0, ignore_index = True)

del skins_history["Unnamed: 0"] # deletes the column name generated

In [ ]:
skins_history = skins_history.drop_duplicates()
skins_history.columns = index
skins_history.to_excel("datasets/all_skins.xlsx")

In [ ]:
# turns all sticker files into one big data frame
path = r"D:\Code\steam_market_tracker\stickerss_query"
all_files = glob.glob(path + "/*.xlsx")

li = []

for filename in all_files:
    df = pd.read_excel(filename, index_col = None, header = 0)
    li.append(df)

stickers_history = pd.concat(li, axis = 0, ignore_index = True)

del stickers_history["Unnamed: 0"] # deletes the column name generated

In [61]:
stickers_history = stickers_history.drop_duplicates()
stickers_history.columns = index
stickers_history.to_excel("datasets/all_stickers.xlsx")

In [25]:
# turns all other files into one big data frame
path = r"D:\Code\steam_market_tracker\others_query"
all_files = glob.glob(path + "/*.xlsx")

li = []

for filename in all_files:
    df = pd.read_excel(filename, index_col = None, header = 0)
    li.append(df)

others_history = pd.concat(li, axis = 0, ignore_index = True)

del others_history["Unnamed: 0"] # deletes the column name generated

In [56]:
others_history = others_history.drop_duplicates()
others_history.columns = index
others_history.to_excel("datasets/all_others.xlsx")

## Other Functions

The following function was used to clean up data as I originally had forgotten to add a collection and quality column.

In [59]:
# originally forgot to add collection and quality name 
# ... oops
# writes the proper weapon and collection by iterating through the all skins list and comparing it to the price history dataframe
# Is used prior to indexing the dataframe

frame.insert(1, "Collection", None)
frame.insert(2, "Quality", None)

for index, row in all_skins.iterrows():
    weapon = row["Weapon"]
    collection = row["Collection"]
    skin = row["Skin"]
    quality = row["Quality"]
    certain_skin_index = frame.index[(frame["Weapon"] == weapon) & (frame["Skin"] == skin)].tolist()

    for x in certain_skin_index:
        frame.iat[x, 1] = collection
        frame.iat[x, 2] = quality